# Loading the DataSet

In [1]:
from pyspark.sql import SparkSession
# Set log level to ERROR (will suppress warnings)


spark = SparkSession.builder.appName('CustomerTransactionCaseStudy').getOrCreate()
spark.sparkContext.setLogLevel("ERROR")


df_large = spark.read.csv('./source/customer_transactions.csv', header=True, inferSchema=True)
df_large.show(10)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/30 20:51:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/30 20:51:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
                                                                                

+--------------------+-------------+----------+------------+-----------+
|         customer_id|purchase_date|product_id|amount_spent|   location|
+--------------------+-------------+----------+------------+-----------+
|cec8ce71-f66c-405...|   2025-02-19|   prod_26|       86.82|    Phoenix|
|9f64fb87-ba35-4d2...|   2024-11-30|   prod_75|      420.97|   New York|
|fc370ee9-49a5-4c6...|   2024-10-12|   prod_33|      210.12|   New York|
|f59526e3-961a-403...|   2024-09-24|   prod_92|      141.91|    Chicago|
|530e6b90-7f0b-494...|   2023-04-11|   prod_53|      306.97|    Houston|
|cf907b49-a479-454...|   2024-06-27|    prod_5|      447.03|    Chicago|
|7be46435-3259-428...|   2024-09-13|   prod_54|      163.58|    Chicago|
|381ff09f-13e4-4fe...|   2024-07-25|   prod_54|      201.32|    Chicago|
|76629f06-a3e4-4c6...|   2023-12-19|   prod_65|      281.95|   New York|
|fc9e962c-7337-40b...|   2024-01-11|   prod_69|      271.06|Los Angeles|
+--------------------+-------------+----------+----

# Data Exploration
- check the schema and inspect some basic statistics to ensure the data is consistent

In [2]:
df_large.printSchema()
df_large.select('customer_id', 'purchase_date', 'amount_spent').describe().show()

#Check data distribution
df_large.groupBy('location').count().show()

root
 |-- customer_id: string (nullable = true)
 |-- purchase_date: date (nullable = true)
 |-- product_id: string (nullable = true)
 |-- amount_spent: double (nullable = true)
 |-- location: string (nullable = true)



+-------+--------------------+------------------+
|summary|         customer_id|      amount_spent|
+-------+--------------------+------------------+
|  count|             5000000|           5000000|
|   mean|                NULL|254.97529644199867|
| stddev|                NULL| 141.4397487898142|
|    min|000019ea-d38b-42d...|              10.0|
|    max|ffffed99-a43a-479...|             500.0|
+-------+--------------------+------------------+

+-----------+-------+
|   location|  count|
+-----------+-------+
|    Phoenix| 999528|
|Los Angeles|1000560|
|    Chicago|1000170|
|    Houston| 998853|
|   New York|1000889|
+-----------+-------+



# Data Cleaning
- Remove duplicates
- Drop rows where amount_spent is missing


In [3]:
df_clean = df_large.dropDuplicates()

df_clean = df_clean.filter(df_clean.amount_spent.isNotNull())
df_clean.printSchema()

from pyspark.sql.functions import col, to_date
df_clean = df_clean.withColumn('purchase_date', to_date(col('purchase_date'), 'yyyy-MM-dd'))
df_clean.show(10)

df_clean.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- purchase_date: date (nullable = true)
 |-- product_id: string (nullable = true)
 |-- amount_spent: double (nullable = true)
 |-- location: string (nullable = true)



[Stage 9:====================================>                      (5 + 3) / 8]

+--------------------+-------------+----------+------------+-----------+
|         customer_id|purchase_date|product_id|amount_spent|   location|
+--------------------+-------------+----------+------------+-----------+
|6f8347e8-6b5b-493...|   2024-03-30|   prod_41|      229.62|    Phoenix|
|dcec3b12-ee58-424...|   2024-03-28|   prod_70|      349.39|Los Angeles|
|6e0f4818-9e7a-4bb...|   2024-06-27|   prod_84|        15.7|    Phoenix|
|49f2d91e-0952-460...|   2025-02-12|   prod_48|       294.4|   New York|
|5a46e3b7-dda1-4d4...|   2025-01-16|   prod_77|       256.9|    Houston|
|f78de9d8-9bd5-412...|   2024-06-13|  prod_100|      349.53|Los Angeles|
|c9191967-57b2-463...|   2024-05-26|    prod_5|      278.71|    Phoenix|
|38bf8534-6232-459...|   2023-09-19|   prod_24|       12.65|Los Angeles|
|c3c17991-4200-438...|   2024-08-25|   prod_64|      388.06|    Chicago|
|4e5eb1e4-669e-48a...|   2023-12-12|   prod_89|      148.77|    Houston|
+--------------------+-------------+----------+----

# Data Aggregation
- Perform some aggregations to compute key metrics.

In [4]:
import time
from pyspark.sql.functions import sum

start_time = time.time()
customer_summary = df_clean.groupBy('customer_id').agg(
    sum('amount_spent').alias('total_spent')
)
customer_summary.show(5)
end_time = time.time()
print(f'Execution Time : {end_time - start_time:.2f} seconds')

[Stage 14:======>                                                   (1 + 8) / 9]

+--------------------+-----------+
|         customer_id|total_spent|
+--------------------+-----------+
|7267fa3f-19dc-48e...|    1790.26|
|3cf01753-5c92-40f...|    2566.46|
|56f7f876-6bdc-44e...|    4374.74|
|660f1afe-8254-4cf...|     1714.3|
|964c2d71-21f0-4b9...|    3951.38|
+--------------------+-----------+
only showing top 5 rows

Execution Time : 5.79 seconds


### Challenge: Large Aggregation Operations
* Aggregations can cause shuffling of the data, which is expensive when working with large datasets
* We can use partitioning or bucketing to minimize the shuffle during the aggregation process

In [5]:
import time
from pyspark.sql.functions import sum

# repartitioning the data into 100 partitions in a way that all rows with the same value for customer_id are likely to end up in the same partition
df_clean_repartitioned = df_clean.repartition(100, 'customer_id') 

start_time = time.time()
customer_summary = df_clean_repartitioned.groupBy('customer_id').agg(
    sum('amount_spent').alias('total_spent')
)
customer_summary.show(5)
end_time = time.time()
print(f'Execution Time : {end_time - start_time:.2f} seconds')

[Stage 20:>                                                         (0 + 8) / 9]

+--------------------+------------------+
|         customer_id|       total_spent|
+--------------------+------------------+
|782d96d7-0e12-48e...|           2648.31|
|4fe6223b-260b-40f...|           2133.73|
|960ac431-6fc2-44e...|           2189.49|
|7267fa3f-19dc-48e...|           1790.26|
|3cf01753-5c92-40f...|2566.4600000000005|
+--------------------+------------------+
only showing top 5 rows

Execution Time : 5.54 seconds


# Identifying Top Customers

In [6]:
top_customers = customer_summary.orderBy('total_spent', ascending=False).limit(10)
top_customers.show(5)

[Stage 26:===================>                                      (3 + 6) / 9]

+--------------------+-----------------+
|         customer_id|      total_spent|
+--------------------+-----------------+
|1cf7e2ba-ccfa-4c1...|8160.939999999998|
|27e5457b-65ff-44d...|8114.209999999999|
|0a185776-4a95-423...|          7557.26|
|9c4ca216-540a-435...|          7484.98|
|f9c8b5c3-055a-49f...|7409.340000000001|
+--------------------+-----------------+
only showing top 5 rows



# Joining with Product and Demographic Data

In [13]:
import time
product_info = spark.read.csv('./source/product_info.csv', header=True, inferSchema=True)
customer_info = spark.read.csv('./source/customer_info.csv', header=True, inferSchema=True)

start_time = time.time()
df_joined = df_clean.join(product_info, "product_id")
df_full = df_joined.join(customer_info, "customer_id")

df_full.show(10)
end_time = time.time()
print(f"Execution time : {end_time - start_time:.2f} seconds")

[Stage 94:================================>                         (5 + 4) / 9]

+--------------------+----------+-------------+------------+--------+-------------+---------------+------+--------------+--------------------+---+-------------+
|         customer_id|product_id|purchase_date|amount_spent|location| product_name|       category| price|          name|               email|age|         city|
+--------------------+----------+-------------+------------+--------+-------------+---------------+------+--------------+--------------------+---+-------------+
|0003b460-ca4e-4e6...|   prod_62|   2024-02-24|      130.11|New York|        Shoes|       Clothing|213.04|Henry Espinoza|scottshawn@exampl...| 27|East Jennifer|
|0003b460-ca4e-4e6...|   prod_49|   2025-02-14|       465.9|New York|      Sweater|       Clothing|752.15|Henry Espinoza|scottshawn@exampl...| 27|East Jennifer|
|0003b460-ca4e-4e6...|   prod_94|   2024-01-01|      288.45|New York|        Shoes|       Clothing| 44.21|Henry Espinoza|scottshawn@exampl...| 27|East Jennifer|
|0003b460-ca4e-4e6...|   prod_74| 

# Optimizations during Join
- Joins can cause shuffling and are often the slowest operation when processing large datasets. We can optimize joins by:

  - Broadcasting the smaller `product_info` dataframe


In [14]:
import time
from pyspark.sql.functions import broadcast

product_info = spark.read.csv('./source/product_info.csv', header=True, inferSchema=True)
customer_info = spark.read.csv('./source/customer_info.csv', header=True, inferSchema=True)

start_time = time.time()
df_joined = df_clean.join(broadcast(product_info), "product_id")
df_full = df_joined.join(customer_info, "customer_id")

df_full.show(10)
end_time = time.time()
print(f"Execution time : {end_time - start_time :.2f} seconds")

[Stage 107:======>                                                  (1 + 8) / 9]

+--------------------+----------+-------------+------------+--------+-------------+---------------+------+--------------+--------------------+---+-------------+
|         customer_id|product_id|purchase_date|amount_spent|location| product_name|       category| price|          name|               email|age|         city|
+--------------------+----------+-------------+------------+--------+-------------+---------------+------+--------------+--------------------+---+-------------+
|0003b460-ca4e-4e6...|   prod_62|   2024-02-24|      130.11|New York|        Shoes|       Clothing|213.04|Henry Espinoza|scottshawn@exampl...| 27|East Jennifer|
|0003b460-ca4e-4e6...|   prod_49|   2025-02-14|       465.9|New York|      Sweater|       Clothing|752.15|Henry Espinoza|scottshawn@exampl...| 27|East Jennifer|
|0003b460-ca4e-4e6...|   prod_94|   2024-01-01|      288.45|New York|        Shoes|       Clothing| 44.21|Henry Espinoza|scottshawn@exampl...| 27|East Jennifer|
|0003b460-ca4e-4e6...|   prod_74| 

# Writing Results
- After performing all the necessary transformations, write the results back to storage.
- It's critical to optimize the write process to avoid generating too many small files

In [15]:
df_full.coalesce(1).write.mode('overwrite').parquet('./target/fully_joined_dataset.parquet')

# Monitoring and Profiling

#### Understanding Plan
Parsed Logical Plan - shows the SQL-like plan before optimization; Displays operations like JOIN, FILTER, PROJECT, AGGREGATE

Analyzed Logical Plan - checks column names and types; Ensures column `customer_id` is valid and `amount_spent` is `double`, etc

Optimized Logical Plan - applies optimizations like: Predicate pushdown (filters only), Broadcast join (small tables sent to all workers), Column pruning (removes unused columns)

Physical Plan - Spark physically executes the query. Uses: BroadcastHashJoin, SortMergeJoin, Exchange operations
    - Displays `partitioning strategy` and `file scan details`

In [19]:
df_full.explain(True)

== Parsed Logical Plan ==
'Join UsingJoin(Inner, [customer_id])
:- Project [product_id#19, customer_id#17, purchase_date#219, amount_spent#20, location#21, product_name#1035, category#1036, price#1037]
:  +- Join Inner, (product_id#19 = product_id#1034)
:     :- Project [customer_id#17, to_date(purchase_date#18, Some(yyyy-MM-dd), Some(America/Los_Angeles), false) AS purchase_date#219, product_id#19, amount_spent#20, location#21]
:     :  +- Filter isnotnull(amount_spent#20)
:     :     +- Deduplicate [purchase_date#18, location#21, customer_id#17, amount_spent#20, product_id#19]
:     :        +- Relation [customer_id#17,purchase_date#18,product_id#19,amount_spent#20,location#21] csv
:     +- ResolvedHint (strategy=broadcast)
:        +- Relation [product_id#1034,product_name#1035,category#1036,price#1037] csv
+- Relation [customer_id#1059,name#1060,email#1061,age#1062,city#1063] csv

== Analyzed Logical Plan ==
customer_id: string, product_id: string, purchase_date: date, amount_spent

# SUMMARY

### Challenges Faced:
Data Shuffling: Shuffling happens during operations like groupBy and join. This can cause performance bottlenecks. Optimization: Partitioning the data appropriately and using broadcast joins for smaller datasets.

Memory Usage: Large aggregations can overwhelm memory. Optimization: Use partitioning and caching appropriately to ensure data fits within the memory.

File Handling: Writing large datasets may result in too many small files. Optimization: Use .coalesce() to reduce the number of output files.



### In this case study, we:

Loaded and cleaned large datasets.

Aggregated the data efficiently using partitioning.

Performed joins using broadcast strategies.

Written optimized results back to storage in a performant way.